In [53]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [54]:
with open('corpus.txt') as f:
    mylist = f.read().splitlines()

In [55]:
for i in range(len(mylist)):
  mylist[i] += ' <e>'

In [56]:
tokenizer = Tokenizer(filters='', lower=False)
tokenizer.fit_on_texts(mylist)
total_words = len(tokenizer.word_index) + 1
print(total_words)
input_sequences = []
incr = 0
for line in mylist:
  if incr % 2 == 0:
    temp_seq = line
  else :
    token_list = tokenizer.texts_to_sequences([temp_seq + ' '+line])[0]
    input_sequences.append(token_list)
  incr+=1

34223


In [57]:
from tensorflow.keras.preprocessing import sequence
input_sequences = np.asarray(input_sequences)
X_train_list = []
Y_train_list = []
for j in range(len(input_sequences)):
  for i in range(len(input_sequences[j])-1):
    X_train_list.append(input_sequences[j][:i+1])
    Y_train_list.append(input_sequences[j][i+1])
maxlen = 0
for seq in X_train_list:
  if len(seq)>maxlen: maxlen = len(seq)
X_train_list = sequence.pad_sequences(X_train_list, maxlen = maxlen)
X_test_list = X_train_list[320003:]
Y_test_list = Y_train_list[320003:]
X_train_list = X_train_list[:320003]
Y_train_list = Y_train_list[:320003]
predictors, label = np.asarray(X_train_list), np.asarray(Y_train_list)
X_test, Y_test = np.asarray(X_test_list), np.asarray(Y_test_list)
print(predictors.shape)
print(label.shape)
print(X_test.shape)
print(Y_test.shape)

(320003, 28)
(320003,)
(51252, 28)
(51252,)


In [58]:
model = Sequential(
        [
         Embedding(total_words, 64, input_length=maxlen),
         LSTM(256, return_sequences=True), Dropout(0.2),
         LSTM(256),
         Dense(total_words, activation='softmax')
        ]
    )
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 28, 64)            2190272   
_________________________________________________________________
lstm_4 (LSTM)                (None, 28, 256)           328704    
_________________________________________________________________
dropout_2 (Dropout)          (None, 28, 256)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_2 (Dense)              (None, 34223)             8795311   
Total params: 11,839,599
Trainable params: 11,839,599
Non-trainable params: 0
_________________________________________________________________
None


In [59]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(predictors, label, batch_size=64, epochs=10, validation_data=(X_test, Y_test))

Epoch 1/6
5001/5001 [==============================] - 160s 32ms/step - loss: 6.8037 - accuracy: 0.1370 - val_loss: 6.4772 - val_accuracy: 0.1663
Epoch 2/6
5001/5001 [==============================] - 160s 32ms/step - loss: 5.8170 - accuracy: 0.1689 - val_loss: 6.2964 - val_accuracy: 0.1741
Epoch 3/6
5001/5001 [==============================] - 160s 32ms/step - loss: 5.3147 - accuracy: 0.1818 - val_loss: 6.2563 - val_accuracy: 0.1768
Epoch 4/6
5001/5001 [==============================] - 161s 32ms/step - loss: 4.9662 - accuracy: 0.1932 - val_loss: 6.2948 - val_accuracy: 0.1853
Epoch 5/6
5001/5001 [==============================] - 160s 32ms/step - loss: 4.6962 - accuracy: 0.2018 - val_loss: 6.4092 - val_accuracy: 0.1837
Epoch 6/6
5001/5001 [==============================] - 160s 32ms/step - loss: 4.4676 - accuracy: 0.2092 - val_loss: 6.5294 - val_accuracy: 0.1862


In [69]:
pattern = predictors[946]

In [70]:
# --- note_l: is the list of notes (integer number) over time
note_l = [k for k in pattern]
# --- prediction_l: is the list of output-vectors (float numbers \in [0,1]) of the network over time
prediction_l = []
# --- Generate T_y_generated notes
for note_index in range(50):
        # --- START CODE HERE (03)
        prediction = model.predict(np.expand_dims(pattern, 0))
        prediction_l.append(prediction)
        index = np.argmax(prediction[0])
        note_l.append(index)
        pattern = np.concatenate((pattern[1:],[index]), axis=0)
        # --- END CODE HERE

In [71]:
print(tokenizer.sequences_to_texts([note_l]))

['L’impie Achab détruit, et de votre propre <e> Que le ciel est pour moi que je vous ai passé. <e> Et je ne veux pas que je ne puis souffrir. <e> Et je ne veux pas que je ne puis souffrir. <e> Et je ne veux pas que je ne puis souffrir. <e> Et je']


**Nouvelle méthode, 10 epochs, pas de batchs**

Oui, je ne puis savoir que vous ne régnez pas.

Ah ! Si vous le croyez, et vous ne pouvez douter,

De ce que je vous plaît, que je vous ai donné ?

Mais je ne puis pas voir un autre que je meure.

Ah ! Si vous le croyez, et vous ne pouvez pas ?

Célébrer à mes yeux la fortune éternelle ?

Mais je ne puis savoir que vous ne régnez pas.

Ah ! Si vous le croyez, et vous ne pouvez rien croire,

Et je ne puis plus voir un autre que je crains.

Je ne vous pas pas dit que je vous ai donné ?

**batch-size 64**

Célébrer en mourant je ferais l’équité.

Je ne sais que je hais que je suis élevé,

Que vous ne l’attaquiez qu’afin de sa vengeance. 

Et je ne puis plus de vaincre et de la ville ; 

Mais je ne veux pas que je ne puis comprendre

Que vous ne craindrons point de ce que je désire ?

Oui, je ne puis souffrir que je suis outragé.

Que je suis redevable à ce triste soupir.

Et je ne veux pas que je vous veux régner.

Je ne sais que vous ne me verra plus rien.

Je ne puis que trop, je ne puis que je hais.

Que je ne veux pas que je vous dois régner.

**20 epochs**

Oui, je ne puis souffrir que je vous ai parlé,

Et je ne savais pas que je doive survivre.

Non non, je voudrai, le soin de mon époux ?

Et je ne savais pas que je vous le promets.

Célébrer en ces lieux le pontife inspiré

De la terre, d’Amurat cache son ombre,

Et je fuirai les jours de Parjure, et de gloire,

Et je ne voudrais pas de ce complot funeste,

Je ne puis que mon coeur teint de ce côté.